## Partie 2 – Traitement de texte (Text Preprocessing)

# OBJECTIFS
    - Nettoyer et transformer les tweets pour la modélisation
    - Réduire la dimensionnalité du corpus
    - Extraire une version épurée et utile du texte

# Étapes à suivre

- Écrire une fonction pour :
    - Tokeniser un tweet
    - Supprimer la ponctuation, les chiffres, les mots courts (< 3 lettres)
    - Supprimer les stopwords
    - Appliquer stemming ou lemmatisation
- Reconstituer un corpus nettoyé
- Analyser :
    - Le nombre total de tokens
    - Le nombre de tokens uniques
    - Le nombre de tokens apparaissant une seule fois
- Visualiser les tokens les plus fréquents (WordCloud)